In [ ]:
import torch
import torch.nn as nn
import torch.fft as fft

class FractionalDiffusionLayer(nn.Module):
    def __init__(self, beta):
        super(FractionalDiffusionLayer, self).__init__()
        self.beta = beta

    def forward(self, x):
        # Apply FFT to compute fractional Laplacian in Fourier space
        x_hat = fft.fft2(x)
        k = torch.fft.fftfreq(x.size(-1))  # Frequency components
        k_norm = torch.norm(k, dim=-1)  # Compute |k| norm
        frac_lap = -torch.pow(k_norm, self.beta)  # Fractional operator
        x_hat = frac_lap * x_hat
        return fft.ifft2(x_hat).real

# Example neural network with fractional diffusion
class FractionalDiffusionNet(nn.Module):
    def __init__(self, beta):
        super(FractionalDiffusionNet, self).__init__()
        self.frac_layer = FractionalDiffusionLayer(beta)
        self.fc = nn.Linear(128, 10)  # Fully connected layer

    def forward(self, x):
        x = self.frac_layer(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

In [ ]:
import torch
import torch.nn as nn
import torch.fft as fft

class FractionalDiffusionLayer(nn.Module):
    """
    Fractional Diffusion Layer applying fractional Laplacian in Fourier space.
    """
    def __init__(self, beta, input_size):
        super(FractionalDiffusionLayer, self).__init__()
        self.beta = beta  # Fractional order
        self.input_size = input_size  # Expected input shape (H, W)
        self._create_frequency_grid()

    def _create_frequency_grid(self):
        """
        Create a frequency grid for computing the fractional Laplacian.
        """
        h, w = self.input_size
        k_x = torch.fft.fftfreq(h).view(-1, 1)  # Frequency components in x
        k_y = torch.fft.fftfreq(w).view(1, -1)  # Frequency components in y
        self.k_norm = torch.sqrt(k_x**2 + k_y**2)  # Compute |k|

    def forward(self, x):
        """
        Forward pass applying fractional Laplacian.
        """
        # Apply FFT
        x_fft = fft.fft2(x)
        x_fft_shifted = fft.fftshift(x_fft)  # Shift zero frequency to the center

        # Compute fractional Laplacian
        frac_lap = -torch.pow(self.k_norm, self.beta).to(x.device)  # |k|^beta
        frac_lap[torch.isnan(frac_lap)] = 0  # Avoid NaNs for zero frequencies
        x_filtered = frac_lap * x_fft_shifted

        # Inverse FFT
        x_filtered = fft.ifftshift(x_filtered)  # Undo shift
        x_filtered = fft.ifft2(x_filtered).real
        return x_filtered


class FractionalDiffusionNet(nn.Module):
    """
    Neural Network with Fractional Diffusion Layer.
    """
    def __init__(self, beta, input_size, num_classes):
        super(FractionalDiffusionNet, self).__init__()
        self.frac_layer = FractionalDiffusionLayer(beta, input_size)
        self.conv = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(16 * (input_size[0] // 2) * (input_size[1] // 2), num_classes)

    def forward(self, x):
        x = self.frac_layer(x)
        x = x.unsqueeze(1)  # Add channel dimension for Conv2D
        x = self.pool(torch.relu(self.conv(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x

# Hyperparameters
beta = 1.5  # Fractional order
input_size = (32, 32)  # Input dimensions (Height, Width)
num_classes = 10  # For classification tasks (e.g., MNIST)

# Model instantiation
model = FractionalDiffusionNet(beta, input_size, num_classes)
print(model)

FractionalDiffusionNet(
  (frac_layer): FractionalDiffusionLayer()
  (conv): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=4096, out_features=10, bias=True)
)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.fft as fft


# Fractional Diffusion Layer
class FractionalDiffusionLayer(nn.Module):
    def __init__(self, beta, input_size):
        super(FractionalDiffusionLayer, self).__init__()
        self.beta = beta  # Fractional order
        self.input_size = input_size  # Expected input shape (H, W)
        self._create_frequency_grid()

    def _create_frequency_grid(self):
        h, w = self.input_size
        k_x = torch.fft.fftfreq(h).view(-1, 1)  # Frequency components in x
        k_y = torch.fft.fftfreq(w).view(1, -1)  # Frequency components in y
        self.k_norm = torch.sqrt(k_x**2 + k_y**2)  # Compute |k|

    def forward(self, x):
        # Input shape: [batch_size, height, width]
        x_fft = fft.fft2(x)  # Shape: [batch_size, height, width]
        x_fft_shifted = fft.fftshift(x_fft)  # Shift to center

        # Compute fractional Laplacian
        frac_lap = -torch.pow(self.k_norm, self.beta).to(x.device)  # Shape: [height, width]
        frac_lap[torch.isnan(frac_lap)] = 0
        x_filtered = frac_lap * x_fft_shifted

        # Inverse FFT
        x_filtered = fft.ifftshift(x_filtered)
        x_filtered = fft.ifft2(x_filtered).real  # Real part of inverse FFT
        return x_filtered  # Output shape: [batch_size, height, width]


# Fractional Diffusion Neural Network
class FractionalDiffusionNet(nn.Module):
    def __init__(self, beta, input_size, num_classes):
        super(FractionalDiffusionNet, self).__init__()
        self.frac_layer = FractionalDiffusionLayer(beta, input_size)
        self.conv = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)  # Conv2D layer
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling
        self.fc = nn.Linear(16 * (input_size[0] // 2) * (input_size[1] // 2), num_classes)  # Fully connected

    def forward(self, x):
        print("Input shape:", x.shape)  # Debugging: Input shape
        x = x.squeeze(1)  # Remove channel dimension if it exists
        x = self.frac_layer(x)  # Apply fractional diffusion
        print("After fractional layer:", x.shape)  # Debugging: Fractional diffusion layer
        x = x.unsqueeze(1)  # Add channel dimension for Conv2D
        print("After unsqueeze:", x.shape)  # Debugging: Before Conv2D
        x = self.pool(torch.relu(self.conv(x)))  # Conv2D and pooling
        print("After Conv2D and pooling:", x.shape)  # Debugging: After Conv2D
        x = x.view(x.size(0), -1)  # Flatten for fully connected layer
        print("After flattening:", x.shape)  # Debugging: Before fully connected
        x = self.fc(x)  # Fully connected layer
        return x

# Training and Testing Loop
def train_and_test():
    # Hyperparameters
    beta = 1.5
    input_size = (32, 32)
    num_classes = 10
    batch_size = 64
    num_epochs = 5
    learning_rate = 0.001

    # Data Preparation
    transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Model, Loss, and Optimizer
    model = FractionalDiffusionNet(beta=beta, input_size=input_size, num_classes=num_classes)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training Loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Testing Loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


# Run the Training and Testing Loop
if __name__ == "__main__":
    train_and_test()

Streaming output truncated to the last 5000 lines.
After unsqueeze: torch.Size([64, 1, 32, 32])
After Conv2D and pooling: torch.Size([64, 16, 16, 16])
After flattening: torch.Size([64, 4096])
Input shape: torch.Size([64, 1, 32, 32])
After fractional layer: torch.Size([64, 32, 32])
After unsqueeze: torch.Size([64, 1, 32, 32])
After Conv2D and pooling: torch.Size([64, 16, 16, 16])
After flattening: torch.Size([64, 4096])
Input shape: torch.Size([64, 1, 32, 32])
After fractional layer: torch.Size([64, 32, 32])
After unsqueeze: torch.Size([64, 1, 32, 32])
After Conv2D and pooling: torch.Size([64, 16, 16, 16])
After flattening: torch.Size([64, 4096])
Input shape: torch.Size([64, 1, 32, 32])
After fractional layer: torch.Size([64, 32, 32])
After unsqueeze: torch.Size([64, 1, 32, 32])
After Conv2D and pooling: torch.Size([64, 16, 16, 16])
After flattening: torch.Size([64, 4096])
Input shape: torch.Size([64, 1, 32, 32])
After fractional layer: torch.Size([64, 32, 32])
After unsqueeze: torch.Si

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.fft as fft


# Fractional Diffusion Layer
class FractionalDiffusionLayer(nn.Module):
    def __init__(self, beta, input_size):
        super(FractionalDiffusionLayer, self).__init__()
        self.beta = beta  # Fractional order
        self.input_size = input_size  # Expected input shape (H, W)
        self._create_frequency_grid()

    def _create_frequency_grid(self):
        h, w = self.input_size
        k_x = torch.fft.fftfreq(h).view(-1, 1)  # Frequency components in x
        k_y = torch.fft.fftfreq(w).view(1, -1)  # Frequency components in y
        self.k_norm = torch.sqrt(k_x**2 + k_y**2)  # Compute |k|

    def forward(self, x):
        # Input shape: [batch_size, height, width]
        x_fft = fft.fft2(x)  # Shape: [batch_size, height, width]
        x_fft_shifted = fft.fftshift(x_fft)  # Shift to center

        # Compute fractional Laplacian
        frac_lap = -torch.pow(self.k_norm, self.beta).to(x.device)  # Shape: [height, width]
        frac_lap[torch.isnan(frac_lap)] = 0
        x_filtered = frac_lap * x_fft_shifted

        # Inverse FFT
        x_filtered = fft.ifftshift(x_filtered)
        x_filtered = fft.ifft2(x_filtered).real  # Real part of inverse FFT
        return x_filtered  # Output shape: [batch_size, height, width]


# Fractional Diffusion Neural Network
class FractionalDiffusionNet(nn.Module):
    def __init__(self, beta, input_size, num_classes):
        super(FractionalDiffusionNet, self).__init__()
        self.frac_layer = FractionalDiffusionLayer(beta, input_size)
        self.conv = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)  # Conv2D layer
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling
        self.fc = nn.Linear(16 * (input_size[0] // 2) * (input_size[1] // 2), num_classes)  # Fully connected

    def forward(self, x):
        x = x.squeeze(1)  # Remove channel dimension if it exists
        x = self.frac_layer(x)  # Apply fractional diffusion
        x = x.unsqueeze(1)  # Add channel dimension for Conv2D
        x = self.pool(torch.relu(self.conv(x)))  # Conv2D and pooling
        x = x.view(x.size(0), -1)  # Flatten for fully connected layer
        x = self.fc(x)  # Fully connected layer
        return x


# Training and Testing Loop
def train_and_test():
    # Hyperparameters
    beta = 1.5
    input_size = (32, 32)
    num_classes = 10
    batch_size = 64
    num_epochs = 5
    learning_rate = 0.001

    # Data Preparation (FashionMNIST)
    transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
    test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

    # Train on a portion of the dataset
    partial_train_size = int(0.1 * len(train_dataset))
    train_dataset, _ = torch.utils.data.random_split(train_dataset, [partial_train_size, len(train_dataset) - partial_train_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Model, Loss, and Optimizer
    model = FractionalDiffusionNet(beta=beta, input_size=input_size, num_classes=num_classes)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training Loop
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

    # Testing Loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


# Run the Training and Testing Loop
if __name__ == "__main__":
    train_and_test()                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             nnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn

100%|██████████| 26.4M/26.4M [00:01<00:00, 16.6MB/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 270kB/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 4.94MB/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 12.2MB/s]


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Epoch [1/5], Loss: 1.0188
Epoch [2/5], Loss: 0.5928
Epoch [3/5], Loss: 0.4997
Epoch [4/5], Loss: 0.4624
Epoch [5/5], Loss: 0.4189
Test Accuracy: 82.50%


**Core Idea: Image Classification with Fractional Diffusion**

The code implements a deep learning model for image classification. The unique aspect is that it incorporates **fractional diffusion** into the neural network architecture.  Traditional convolutional neural networks (CNNs) learn local patterns in images. This code attempts to enhance the network's ability to capture more global, long-range dependencies in the data using the concept of fractional diffusion, which is inspired by physical diffusion processes.

**What is Fractional Diffusion?**

Imagine dropping a drop of ink into water. The ink diffuses outward, spreading over time. This is a standard diffusion process. Fractional diffusion is a generalization of this idea, where the spreading behavior is controlled by a fractional derivative (specifically, the fractional Laplacian in this case).

*   **Fractional Laplacian:** The fractional Laplacian is a mathematical operator that extends the concept of the regular Laplacian (which is related to the second derivative). It allows for a more nuanced control over how "diffusion" occurs. The order of the fractional Laplacian (represented by `beta` in the code) determines the nature of the diffusion.
*   **Impact on Image Processing:** When applied to images, fractional diffusion can enhance or suppress certain features based on their spatial frequency. A higher `beta` emphasizes finer details (high-frequency components), while a lower `beta` highlights broader structures (low-frequency components).

**How Fractional Diffusion is Integrated into the Network**

The code defines a `FractionalDiffusionLayer`. This layer performs the following steps:

1.  **FFT (Fast Fourier Transform):** Converts the input image from the spatial domain to the frequency domain.
2.  **Frequency Shifting:** Shifts the zero-frequency component to the center of the spectrum for easier manipulation.
3.  **Fractional Laplacian Application:** Multiplies the frequency representation of the image by the fractional Laplacian operator. This is where the core "fractional diffusion" effect happens. The `k_norm` tensor represents the magnitude of each frequency component, and raising it to the power of `beta` (the fractional order) implements the fractional Laplacian.
4.  **Inverse FFT:** Converts the modified frequency representation back to the spatial domain.

**The `FractionalDiffusionNet` Model**

This class defines the overall neural network architecture:

1.  **Fractional Diffusion Layers:** The network starts with two `FractionalDiffusionLayer`s. These layers apply the fractional diffusion process to the input image, effectively pre-processing the image to highlight features based on the chosen `beta` value.
2.  **Convolutional Layers:** Standard convolutional layers (`nn.Conv2d`) with batch normalization (`nn.BatchNorm2d`) and ReLU activation are used to extract local features, just like in a traditional CNN.
3.  **Pooling Layers:** Max-pooling layers (`nn.MaxPool2d`) downsample the feature maps, reducing their spatial dimensions and increasing the network's receptive field.
4.  **Dropout:** Dropout (`nn.Dropout`) is applied to prevent overfitting by randomly dropping out neurons during training.
5.  **Fully Connected Layer:** A fully connected layer (`nn.Linear`) maps the learned features to the output classes.

**Training and Testing**

The `train_and_test` function handles the training and evaluation of the model:

1.  **Data Loading:** It loads the FashionMNIST dataset, which contains images of clothing items. The data is preprocessed (resized, normalized) and split into training and testing sets.
2.  **Model Initialization:** It creates an instance of the `FractionalDiffusionNet` model.
3.  **Loss Function and Optimizer:** It defines the cross-entropy loss function (for multi-class classification) and the Adam optimizer (with L2 regularization to help prevent overfitting).
4.  **Learning Rate Scheduler:** A learning rate scheduler (`ReduceLROnPlateau`) is used to adjust the learning rate during training based on the validation loss.
5.  **Training Loop:** Iterates through the training data for a specified number of epochs, performing forward and backward passes to update the model's weights.
6.  **Testing Loop:** Evaluates the trained model on the test data to measure its accuracy.

**In Summary**

The code implements a novel neural network architecture for image classification that leverages fractional diffusion to capture long-range dependencies and enhance feature extraction. It combines the power of fractional calculus with standard deep learning techniques (convolution, pooling, etc.) to potentially improve classification performance, especially on datasets where global context is important. The trainable `beta` parameter in the `FractionalDiffusionLayer` allows the network to learn the optimal diffusion behavior for the specific task.